<a href="https://colab.research.google.com/github/Stefano-Previti/Curiosity_driven_exploration_by_self_supervised_prediction/blob/main/Curiosity_driven_exploration_by_Self_supervised_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**▶INITIAL SETUP OF THE ENVIROMENT**

In [ ]:
# Super Mario environment for OpenAI Gym
!pip install gym-super-mario-bros

# Backend for NES emulation
!pip install nes-py

# PyTorch for building and training neural networks
!pip install torch torchvision

# TensorBoard for monitoring training progress
!pip install tensorboard

# Visualization tools for plotting and analysis
!pip install matplotlib


In [3]:
from pyvirtualdisplay import Display
import gym
from gym_super_mario_bros import make
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
from gym.wrappers import FrameStack
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
from PIL import Image
import numpy as np
import cv2
import random
import time

class SuperMarioEnvironment:
    def __init__(self, level_id="SuperMarioBros-1-1-v0", frame_size=(42, 42)):
        """Initialize the Super Mario Environment.
        """
        self.level_id = level_id
        self.env = None
        self.actions = COMPLEX_MOVEMENT
        self.frame_size = frame_size

    def preprocess_frame(self, frame):
        """Preprocess a frame by resizing and converting it to grayscale.
        """
        if isinstance(frame, gym.wrappers.frame_stack.LazyFrames):
            frame = np.array(frame)
        resized_stack = np.array([
            cv2.resize(cv2.cvtColor(f, cv2.COLOR_RGB2GRAY), self.frame_size) for f in frame
        ])
        resized_stack = resized_stack[..., np.newaxis]
        return resized_stack

    def create_environment(self):
        """Create the Super Mario environment with frame stacking."""
        self.env = make(self.level_id)
        self.env = FrameStack(self.env, num_stack=4)
        print(f"Environment for {self.level_id} created.")

    def reset_environment(self):
        """Reset the environment and return the preprocessed initial state."""
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        frame = self.env.reset()
        return self.preprocess_frame(frame)

    def step(self, action):
        """Take an action in the environment.
        """
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        state, reward, done, info = self.env.step(action)
        return self.preprocess_frame(state), reward, done, info

    def render_frame(self):
        """Render the current frame in RGB format."""
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        return self.env.render(mode='rgb_array')

    def close(self):
        """Close the environment."""
        if self.env:
            self.env.close()


In [9]:


# Create the game environment
env = make("SuperMarioBros-1-1-v0")
env = FrameStack(env, num_stack=4)  # Stack 4 consecutive frames

# Get frame dimensions
state = env.reset()
img = env.render(mode='rgb_array')
frame_height, frame_width, _ = img.shape

# Configure video writer
video_filename = "super_mario.mp4"
fps = 30  # Frames per second
video_writer = cv2.VideoWriter(
    video_filename,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (frame_width, frame_height),
)

# Frame buffer
frame_buffer = []
frame_time = 1 / fps  # Time per frame

try:
    start_time = time.time()
    for step in range(1000):  # Number of steps to simulate
        # Perform a random action
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)

        # Capture frame and add to buffer
        img = env.render(mode='rgb_array')
        frame_buffer.append(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

        # Handle resets
        if done:
            print(f"Game Over at step {step}. Restarting...")
            state = env.reset()
            continue  # Skip writing frames during reset

        # Synchronize frame timing
        elapsed_time = time.time() - start_time
        if elapsed_time < frame_time:
            time.sleep(frame_time - elapsed_time)
        start_time = time.time()

    # Write all buffered frames to the video file
    for frame in frame_buffer:
        video_writer.write(frame)

finally:
    env.close()
    video_writer.release()
    print(f"Video saved as {video_filename}")


Video saved as super_mario.mp4
